abalone 데이터를 가지고 딥러닝 분류 작업을 수행

딥러닝에서 중요한 것:
optimizer: 손실값을 최적화하는 기법
loss function: 손실을 어떻게 계산할 것인가
activation function:


Id index를 굳이 넣지 않아도 되는 이유: 차원의 저주가 발생하기 때문.

In [1]:
# 기본 데이터 처리
import pandas as pd
import numpy as np

# 데이터 전처리
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight

# 딥러닝 모델 구축
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical

# 시각화 (선택적)
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# 데이터 로드 및 전처리
df = pd.read_csv('C:/dataset/abalone.csv')
# 1. 클래스 재구성
def categorize_rings(rings):
    if rings <= 5:
        return 0    # 어린 전복
    elif rings <= 10:
        return 1    # 중간 전복
    else:
        return 2    # 성숙 전복

In [3]:
# 데이터 전처리
df['Rings_category'] = df['Rings'].apply(categorize_rings)

In [4]:
# Sex 원-핫 인코딩
df = pd.get_dummies(df, columns=['Sex'])

In [11]:
# 특성과 레이블 분리
X = df.drop(['Rings', 'Rings_category', 'id'], axis=1)
y = df['Rings_category']

In [12]:
# 레이블 원-핫 인코딩
y = to_categorical(y)

In [13]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

In [24]:
# 데이터 스케일링
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [41]:
# 개선된 모델 구조
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(3, activation='softmax')  # 3개 클래스로 변경
])

C:\Users\김경환\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [42]:
# 컴파일
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [43]:
# 클래스 가중치 계산
class_weights = compute_class_weight('balanced', 
                                   classes=np.unique(np.argmax(y_train, axis=1)),
                                   y=np.argmax(y_train, axis=1))
class_weight_dict = dict(enumerate(class_weights))

In [44]:
# 학습
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    class_weight=class_weight_dict,
    verbose=1
)

Epoch 1/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.4080 - loss: 1.1273 - val_accuracy: 0.6054 - val_loss: 0.9303
Epoch 2/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5358 - loss: 0.7625 - val_accuracy: 0.6293 - val_loss: 0.8199
Epoch 3/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5765 - loss: 0.7200 - val_accuracy: 0.6607 - val_loss: 0.7466
Epoch 4/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6098 - loss: 0.6025 - val_accuracy: 0.6652 - val_loss: 0.6921
Epoch 5/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6471 - loss: 0.5724 - val_accuracy: 0.7085 - val_loss: 0.6485
Epoch 6/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6422 - loss: 0.5893 - val_accuracy: 0.6741 - val_loss: 0.6394
Epoch 7/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6656 - loss: 0.5589 - val_accuracy: 0.6906 - val_loss: 0.6413
Epoch 8/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6619 - loss: 0.5477 - val_accuracy: 0.6951 - val_loss